지금 db에서 null값인 데이터를 investing에 검색한 결과 크롤링

In [6]:
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm
import time

from random import uniform, seed
seed(1)
import re


from bs4 import BeautifulSoup

In [2]:
no_isin = pd.read_csv('./no_isin.csv', index_col=0)
no_isin

,ticker,isin
31,JD,mismatch
39,TEAM,mismatch
53,ZIONP,noresult
57,MAR,mismatch
62,NXPI,mismatch
...,...,...
8340,GLO RT WI,noresult
8341,TELZ,noresult
8342,LNFA.U,noresult
8343,BRKB,noresult


In [98]:
ticker_list = pd.read_csv('./ticker_list_null.csv')
ticker_list

,Ticker
0,AACI
1,AACIU
2,AACIW
3,AADI
4,AADR
...,...
7761,ZTEST
7762,ZVIA
7763,ZVV
7764,ZXIET


In [100]:
for ticker in ticker_list['Ticker']:
    print(ticker)
    break

AACI


In [3]:
no_isin_1 = no_isin.iloc[2592:]

In [8]:
def isin_print(ticker):
# ticker를 입력값으로 하여 이동할 url을 찾는다
    url= 'https://api.investing.com/api/search/v2/search?'
    data={
        "q": f"{ticker}"
    }
    header ={
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'origin': 'https://kr.investing.com',
    'referer': 'https://kr.investing.com/search/?q=950130',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
    }
    response = requests.get(url,headers=header,params=data)
    # url 추출
    if response.json()['quotes'] != []:
        detail_url = response.json()['quotes'][0]['url']

        # 찾은 url로 request한다.
        url='https://investing.com'+detail_url
        header ={
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
        }
        response = requests.get(url,headers=header)

        # request로 이동 후 BeautiSoup으로 element값을 통해 isin을 추출한다
        html_doc = response.text
        soup = BeautifulSoup(html_doc, 'html.parser')
        news_tit = soup.findAll("span", {"class":"key-info_dd-numeric__2cYjc"})
        if news_tit != []:
            try:
                prev_close = news_tit[0].text
            except:
                prev_close = np.nan
            
            try:
                margket_cap = news_tit[7].text
            except:
                margket_cap = np.nan
            print(margket_cap)
            try:
                beta = news_tit[-3].text
            except:
                beta = np.nan
                
            try:
                shares = news_tit[-1].text
            except:
                shares = np.nan

            url='https://investing.com'+detail_url+'-ratios'
            header ={
                "User-Agent": "Mozilla/5.0",
                'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
                'accept-encoding': 'gzip, deflate, br',
                'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7'
            }
            response = requests.get(url,headers=header)

            # request로 이동 후 BeautiSoup으로 element값을 통해 isin을 추출한다
            html_doc = response.text

            if re.search('Price to Book', html_doc) != None:
                if re.search('>\d.*?\d<',re.search('Price to Book.*?\d<', html_doc.replace("\n","ss"))[0]) != None:
                    price2book = (re.search('>\d.*?\d<',re.search('Price to Book.*?\d<', html_doc.replace("\n","ss"))[0])[0]).replace("<","").replace(">","")
                else:
                    price2book = np.nan
            else:
                price2book = np.nan

            return ticker, prev_close, margket_cap, shares, beta, price2book

In [286]:
factor = pd.DataFrame(index = ticker_list['Ticker'].iloc[5788:])
for i, ticker in enumerate(ticker_list['Ticker'].iloc[5788:]):
    result = isin_print(ticker) if isin_print(ticker) != None else (ticker, np.nan, np.nan, np.nan, np.nan, np.nan)
    if result == None: result = (ticker, np.nan, np.nan, np.nan, np.nan, np.nan)
    print(i, result)
    factor.loc[ticker, 'Prev_Close_Price'] = result[1]
    factor.loc[ticker, 'Market_Cap'] = result[2]
    factor.loc[ticker, 'Shares'] = result[3]
    factor.loc[ticker, 'Beta'] = result[4]
    factor.loc[ticker, 'Price_To_Book'] = result[5]

0 ('LEV', '27,940', '-29.59%', '-29.59%', '18,939', nan)
1 ('LEV.W', nan, nan, nan, nan, nan)
2 ('LEXI', '3.1', '-0.652', '193,793,551', '1.5', '8.4')
3 ('LFEQ', nan, nan, nan, nan, nan)
4 ('LFG', '2.2', '0.42', '14,112,499', '0.41', '0.65')
5 ('LFT$A', nan, nan, nan, nan, nan)
6 ('LGF.A', nan, nan, nan, nan, nan)
7 ('LGF.B', nan, nan, nan, nan, nan)
8 ('LGH', nan, nan, nan, nan, nan)
9 ('LGL.W', nan, nan, nan, nan, nan)
10 ('LGLV', nan, nan, nan, nan, nan)
11 ('LGOV', nan, nan, nan, nan, nan)
12 ('LGV', nan, nan, nan, nan, nan)
13 ('LGV.U', nan, nan, nan, nan, nan)
14 ('LGV.W', nan, nan, nan, nan, nan)
15 ('LHC', '162.87', '2.38', '31,028,822', '0.48', '3.21')
16 ('LHC.U', nan, nan, nan, nan, nan)
17 ('LHC.W', nan, nan, nan, nan, nan)
18 ('LICY', nan, nan, nan, nan, nan)
19 ('LII', '267.1', '12.95', '35,431,396', '0.951', nan)
20 ('LIT', nan, nan, nan, nan, nan)
21 ('LIV', '64.08', '-0.55', '53,545,733', '0.885', '2.71')
22 ('LKOR', nan, nan, nan, nan, nan)
23 ('LLAP', nan, nan, nan, 

75 ('MAS', '56.29', '3.6', '225,519,663', '1.23', nan)
76 ('MATV', '24.27', '2.49', '54,913,447', '0.886', '1.91')
77 ('MAX', '27.09', '-0.782', '31,330,397', '1.27', '1.63')
78 ('MAYZ', nan, nan, nan, nan, nan)
79 ('MBAC', '2.15', '0.624', '188,771,051', '-1.35', '1.35')
80 ('MBAC.U', nan, nan, nan, nan, nan)
81 ('MBAC.W', nan, nan, nan, nan, nan)
82 ('MBBB', nan, nan, nan, nan, nan)
83 ('MBCC', nan, nan, nan, nan, nan)
84 ('MBI', '12.74', '-7.79', '54,899,913', '1.09', nan)
85 ('MBND', nan, nan, nan, nan, nan)
86 ('MBNE', nan, nan, nan, nan, nan)
87 ('MBOX', '0.00003061', '0.00013406', '0.00013406', '0.00003118', nan)
88 ('MBSC', '26.2', '1.56', '70,189,258', '0.035', '1.29')
89 ('MBSC.U', nan, nan, nan, nan, nan)
90 ('MBSC.W', nan, nan, nan, nan, nan)
91 ('MBSD', nan, nan, nan, nan, nan)
92 ('MCD', '266.58', '8.18', '735,717,271', '0.561', nan)
93 ('MCG', '87.76', '3.98', '24,378,043', '0.791', '2.82')


C:\Users\finset\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


94 ('MCH', nan, nan, nan, nan, nan)
95 ('MCK', '368.56', '874,158', '143,730,455', '0.625', nan)
96 ('MCRO', nan, nan, nan, nan, nan)
97 ('MCW', nan, nan, nan, nan, nan)
98 ('MDEV', nan, nan, nan, nan, nan)
99 ('MDH', nan, nan, nan, nan, nan)
100 ('MDH.U', nan, nan, nan, nan, nan)
101 ('MDH.W', nan, nan, nan, nan, nan)
102 ('MDV', '1.06', '-1.16', '80,021,737', '302,166', '2.54')
103 ('MDV$A', nan, nan, nan, nan, nan)
104 ('MDY', nan, nan, nan, nan, nan)
105 ('MDYG', nan, nan, nan, nan, nan)
106 ('MDYV', nan, nan, nan, nan, nan)
107 ('MEAR', nan, nan, nan, nan, nan)
108 ('MEG', '17.14', '2.66', '304,500,000', '3.01', '1.2')
109 ('MEGI', '292.5', '-17.57', '132,531,580', '0.4', '0.21')
110 ('MEM', '0.031', '0', '792,178,337', '0.34', '2.8')
111 ('MEME', '0.015', '0', '137,622,995', '3.11', nan)
112 ('MENV', nan, nan, nan, nan, nan)
113 ('MER$K', '182', '7.62', '434,777,878', '0.667', '3.05')
114 ('MET$A', nan, nan, nan, nan, nan)
115 ('MET$E', '43.16', '2.74', '36,748,610', '1.15', '1.7

146 ('MIMO.B', nan, nan, nan, nan, nan)
147 ('MIMO.C', nan, nan, nan, nan, nan)
148 ('MIMO.W', nan, nan, nan, nan, nan)
149 ('MINC', nan, nan, nan, nan, nan)
150 ('MINN', '15.11', '0.67', '5,782,386', '0.21', '1.27')
151 ('MINO', '33.64', '2.09', '76,247,585', '0.564', '2.5')
152 ('MINT', nan, nan, nan, nan, nan)
153 ('MINV', '0.017', '0', '328,123,035', '-0.21', '0.56')
154 ('MIO', '803.14', '36.07%', '36.07%', '578.66', nan)
155 ('MIR', '3.02', '0.038', '191,102,118', '0.657', '1.25')
156 ('MIR.W', nan, nan, nan, nan, nan)
157 ('MIT', '11.07', '-1.77', '7,133,575', '0.143', '1.27')
158 ('MIT.U', nan, nan, nan, nan, nan)
159 ('MIT.W', nan, nan, nan, nan, nan)
160 ('MITQ', nan, nan, nan, nan, nan)
161 ('MITT$A', '2.6', '0.082', '3,100,000', '0.054', '0.74')
162 ('MITT$B', nan, nan, nan, nan, nan)
163 ('MITT$C', nan, nan, nan, nan, nan)
164 ('MJ', nan, nan, nan, nan, nan)
165 ('MJIN', nan, nan, nan, nan, nan)
166 ('MJUS', nan, nan, nan, nan, nan)
167 ('MJXL', nan, nan, nan, nan, nan)
16

...

1307 ('STRE', '74.2', '7.32', '367,619,359', '1.61', '1.12')
1308 ('STRE.U', nan, nan, nan, nan, nan)
1309 ('STRE.W', nan, nan, nan, nan, nan)
1310 ('STRY', '220.84', '5.5', '378,320,706', '1.03', '5.27')
1311 ('STRY.W', nan, nan, nan, nan, nan)
1312 ('STT$D', nan, nan, nan, nan, nan)
1313 ('STT$G', nan, nan, nan, nan, nan)
1314 ('STVN', nan, nan, nan, nan, nan)
1315 ('STZ.B', nan, nan, nan, nan, nan)
1316 ('SUAC', nan, nan, nan, nan, nan)
1317 ('SUAC.U', nan, nan, nan, nan, nan)
1318 ('SUAC.W', nan, nan, nan, nan, nan)
1319 ('SUB', nan, nan, nan, nan, nan)
1320 ('SUBS', '0.00000003', '0.00000015', '-66.67%', '0.00000002', nan)
1321 ('SUNL', '35.2', '9.7', '4,200,000', '0.168', nan)
1322 ('SUNL.W', nan, nan, nan, nan, nan)
1323 ('SUP', '6', '-0.963', '27,016,125', '3.96', nan)
1324 ('SUPL', nan, nan, nan, nan, nan)
1325 ('SUSA', nan, nan, nan, nan, nan)
1326 ('SVAL', nan, nan, nan, nan, nan)
1327 ('SVIX', nan, nan, nan, nan, nan)
1328 ('SVOL', '72', '3.47', '102,400,000', '1.67', '0.95

1388 ('TEQI', nan, nan, nan, nan, nan)
1389 ('TFC$I', nan, nan, nan, nan, nan)
1390 ('TFC$O', '33', '0.872', '392,756,513', '0.226', '5.37')
1391 ('TFC$R', nan, nan, nan, nan, nan)
1392 ('TFI', nan, nan, nan, nan, nan)
1393 ('TFJL', nan, nan, nan, nan, nan)
1394 ('TFLO', nan, nan, nan, nan, nan)
1395 ('TFSA', nan, nan, nan, nan, nan)
1396 ('TGH$A', '272', '26.34', '2,879,215', '0.454', '2.47')
1397 ('TGH$B', nan, nan, nan, nan, nan)
1398 ('TGI', '14.95', '-0.352', '64,970,204', '2.66', nan)
1399 ('TGIF', '0.025', '-0.038', '450,699,319', '0.989', '0.94')
1400 ('TGR', '5.16', '0.259', '214,821,181', '0.162', '1.32')
1401 ('TGR.U', nan, nan, nan, nan, nan)
1402 ('TGR.W', nan, nan, nan, nan, nan)
1403 ('TGRW', nan, nan, nan, nan, nan)
1404 ('THCX', nan, nan, nan, nan, nan)
1405 ('THD', nan, nan, nan, nan, nan)
1406 ('THNQ', nan, nan, nan, nan, nan)
1407 ('THQ', '20.97', '2.71', '41,340,446', '0.91', '1.27')
1408 ('THW', '15.49', '1.14', '37,176,392', '0.91', '1.27')
1409 ('THY', '69.65', 

1575 ('UPRO', nan, nan, nan, nan, nan)
1576 ('UPV', nan, nan, nan, nan, nan)
1577 ('UPW', nan, nan, nan, nan, nan)
1578 ('URA', nan, nan, nan, nan, nan)
1579 ('URE', nan, nan, nan, nan, nan)
1580 ('URNM', nan, nan, nan, nan, nan)
1581 ('URTH', nan, nan, nan, nan, nan)
1582 ('URTY', nan, nan, nan, nan, nan)
1583 ('USAI', nan, nan, nan, nan, nan)
1584 ('USB$A', nan, nan, nan, nan, nan)
1585 ('USB$H', nan, nan, nan, nan, nan)
1586 ('USB$P', nan, nan, nan, nan, nan)
1587 ('USB$Q', nan, nan, nan, nan, nan)
1588 ('USB$R', nan, nan, nan, nan, nan)
1589 ('USB$S', nan, nan, nan, nan, nan)
1590 ('USCI', nan, nan, nan, nan, nan)
1591 ('USD', nan, nan, nan, nan, nan)
1592 ('USDU', nan, nan, nan, nan, nan)
1593 ('USEP', nan, nan, nan, nan, nan)
1594 ('USEQ', nan, nan, nan, nan, nan)
1595 ('USER', '1,656', '45.36', '15,870,099', '0.956', '4.75')
1596 ('USFR', nan, nan, nan, nan, nan)
1597 ('USHY', nan, nan, nan, nan, nan)
1598 ('USI', '1.51', '-0.167', '25,188,217', '1.49', '2.53')
1599 ('USL', nan,

1717 ('VPL', nan, nan, nan, nan, nan)
1718 ('VPU', nan, nan, nan, nan, nan)
1719 ('VRAI', nan, nan, nan, nan, nan)
1720 ('VRP', nan, nan, nan, nan, nan)
1721 ('VSCO', nan, nan, nan, nan, nan)
1722 ('VSGX', nan, nan, nan, nan, nan)
1723 ('VSL', '0.00000007', '0.00000031', '-77.42%', '0.00000005', nan)
1724 ('VSLU', nan, nan, nan, nan, nan)
1725 ('VSPY', nan, nan, nan, nan, nan)
1726 ('VSS', nan, nan, nan, nan, nan)
1727 ('VST.A', nan, nan, nan, nan, nan)
1728 ('VT', nan, nan, nan, nan, nan)
1729 ('VTEB', nan, nan, nan, nan, nan)
1730 ('VTEX', nan, nan, nan, nan, nan)
1731 ('VTI', nan, nan, nan, nan, nan)
1732 ('VTOL', '28.16', '0.084', '27,936,165', '1.42', '0.98')
1733 ('VTV', nan, nan, nan, nan, nan)
1734 ('VUG', nan, nan, nan, nan, nan)
1735 ('VUSB', nan, nan, nan, nan, nan)
1736 ('VUSE', nan, nan, nan, nan, nan)
1737 ('VV', nan, nan, nan, nan, nan)
1738 ('VVNT', nan, nan, nan, nan, nan)
1739 ('VVX', '39.49', '-0.254', '30,441,455', '1.1', '3.21')
1740 ('VWID', nan, nan, nan, nan, na

1913 ('XRLV', nan, nan, nan, nan, nan)
1914 ('XRMI', nan, nan, nan, nan, nan)
1915 ('XRT', nan, nan, nan, nan, nan)
1916 ('XSD', nan, nan, nan, nan, nan)
1917 ('XSHD', nan, nan, nan, nan, nan)
1918 ('XSHQ', nan, nan, nan, nan, nan)
1919 ('XSLV', nan, nan, nan, nan, nan)
1920 ('XSMO', nan, nan, nan, nan, nan)
1921 ('XSOE', nan, nan, nan, nan, nan)
1922 ('XSVM', nan, nan, nan, nan, nan)
1923 ('XSW', nan, nan, nan, nan, nan)
1924 ('XTAP', nan, nan, nan, nan, nan)
1925 ('XTJA', nan, nan, nan, nan, nan)
1926 ('XTJL', nan, nan, nan, nan, nan)
1927 ('XTL', nan, nan, nan, nan, nan)
1928 ('XTN', nan, nan, nan, nan, nan)
1929 ('XTOC', nan, nan, nan, nan, nan)
1930 ('XTR', nan, nan, nan, nan, nan)
1931 ('XVOL', nan, nan, nan, nan, nan)
1932 ('XVV', nan, nan, nan, nan, nan)
1933 ('XWEB', nan, nan, nan, nan, nan)
1934 ('XYLD', nan, nan, nan, nan, nan)
1935 ('XYLG', nan, nan, nan, nan, nan)
1936 ('YALA', nan, nan, nan, nan, nan)
1937 ('YANG', nan, nan, nan, nan, nan)
1938 ('YCBD$A', nan, nan, nan, n

In [287]:
factor.to_csv('factor_by_ticker5788_.csv')

In [284]:
ticker_list.iloc[5786]

Ticker    LETB
Name: 5786, dtype: object

In [ ]:
def isin_print(ticker):
# ticker를 입력값으로 하여 이동할 url을 찾는다
    url= 'https://api.investing.com/api/search/v2/search?'
    data={
        "q": f"{ticker}"
    }
    header ={
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'origin': 'https://kr.investing.com',
    'referer': 'https://kr.investing.com/search/?q=950130',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
    }
    response = requests.get(url,headers=header,params=data)
    # url 추출
    
    if response.json().get['quotes'] != []:
        detail_url = response.json().get['quotes'][0].get['url']

        # 찾은 url로 request한다.
        url='https://investing.com'+detail_url
        header ={
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
        }
        response = requests.get(url,headers=header)

        # request로 이동 후 BeautiSoup으로 element값을 통해 isin을 추출한다
        html_doc = response.text
        soup = BeautifulSoup(html_doc, 'html.parser')
        news_tit = soup.findAll("span", {"class":"key-info_dd-numeric__2cYjc"})

        prev_close = news_tit[0].text
        margket_cap = news_tit[7].text
        beta = news_tit[-3].text
        shares = news_tit[-1].text

        url='https://investing.com'+detail_url+'-ratios'
        header ={
            "User-Agent": "Mozilla/5.0",
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7'
        }
        response = requests.get(url,headers=header)

        # request로 이동 후 BeautiSoup으로 element값을 통해 isin을 추출한다
        html_doc = response.text
        soup = BeautifulSoup(html_doc, 'html.parser')
        news_tit = soup.findAll("td", {"class":"child"})
        print(news_tit[4])
        #price2book = str(news_tit[4]).replace('</td>', '').split('<td>')[-2].strip()

        return prev_close, margket_cap, beta, shares#, price2book

In [7]:
isin_new =  pd.Series()
for f_ticker in no_isin_1.ticker:
    try:
        isin_new[f'{f_ticker}'] = isin_print(f_ticker)
    except:
        isin_new[f'{f_ticker}'] = 'no result'
        
    time.sleep(0.4)

C:\Users\finset\AppData\Local\Temp\ipykernel_33588\512730256.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  isin_new =  pd.Series()
C:\Users\finset\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
C:\Users\finset\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
C:\Users\finset\anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
C:\Users\finset\anaconda3\lib

In [8]:
isin_new.to_csv('isin_new_2.csv')

In [10]:
isin_new.value_counts()

no result       750
US46578C1080      2
IL0011796880      2
IT0005452658      1
KYG794711013      1
               ... 
US46132C1071      1
BMG8192H1060      1
US25862D1054      1
US67079X1028      1
US1156372096      1
Length: 1249, dtype: int64

In [129]:
html_doc

' <!DOCTYPE HTML>\n<html dir="ltr" xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" xmlns:schema="http://schema.org/" class="com" lang="en" geo="KR">\n<head>\n<link rel="dns-prefetch" href="https://i-invdn-com.investing.com" />\n<link rel="dns-prefetch" href="https://a-invdn-com.investing.com" />\n<link rel="preconnect" href="https://securepubads.g.doubleclick.net" />\n<link rel="dns-prefetch" href="https://securepubads.g.doubleclick.net" />\n<script async src="https://securepubads.g.doubleclick.net/tag/js/gpt.js"></script>\n<script type="text/javascript">\n    function OptanonWrapper() { }\n\n    // temp for checks\n    </script>\n<style>\n\t.ot-floating-button {\n\t\tz-index:12 !important;\n\t}\n</style>\n<script type="text/javascript">\n        window.helpers = {\n            \'getCookie\': function (k){var v="; "+document.cookie;var p=v.split("; "+k+"=");if(p.length===2)return p.pop().split(";").shift();else return null},\n            \'setCookie\': function (k,v,d){var e=\'\';if

In [130]:
soup_1 = BeautifulSoup(html_doc, 'html.parser')

In [131]:
soup_1

 <!DOCTYPE HTML>

<html class="com" dir="ltr" geo="KR" lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml" xmlns:schema="http://schema.org/">
<head>
<link href="https://i-invdn-com.investing.com" rel="dns-prefetch"/>
<link href="https://a-invdn-com.investing.com" rel="dns-prefetch"/>
<link href="https://securepubads.g.doubleclick.net" rel="preconnect"/>
<link href="https://securepubads.g.doubleclick.net" rel="dns-prefetch"/>
<script async="" src="https://securepubads.g.doubleclick.net/tag/js/gpt.js"></script>
<script type="text/javascript">
    function OptanonWrapper() { }

    // temp for checks
    </script>
<style>
	.ot-floating-button {
		z-index:12 !important;
	}
</style>
<script type="text/javascript">
        window.helpers = {
            'getCookie': function (k){var v="; "+document.cookie;var p=v.split("; "+k+"=");if(p.length===2)return p.pop().split(";").shift();else return null},
            'setCookie': function (k,v,d){var e='';if(typeof d!=='undefined'){var n=n

In [148]:
count = 0
for item in soup_1.findAll():
    
    print('*='*20)
    print(item.get_text())
    
#     if count == 1:
#         # print(item.get_text())
        
    
#     if 'Price to Book' in item.get_text():
#         count = 1
#     else:
#         count = 0

*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=














Mediaalpha Inc (MAX) Financial Ratios























































 



















































Breaking News













Get Actionable Insights with InvestingPro+: Start 7 Day FREE Trial Register here















 






Quotes


All Instrument Types



All Instrument TypesIndicesEquitiesETFsFundsCommoditiesCurrenciesCryptoBondsCertificates 







Please try another search



Search website for: 









Popular News
More


 
Wall St ends higher, Cisco Systems jumps after forecast



 
Cohen Confirms He Sold His Entire Bed Bath & Beyond Stake, Raising Eyebrows



 
Jobless Claims, U.S.-Taiwan Talks, Cohen's Rugpull - What's Moving Markets






Popular Analysis
More


 
Watch What The VIX/VXV Ratio Is Showing



 
3 Tech Stocks Regaining Mojo As Nasdaq Enters New Bull Market



 
Is Apple Heading To $200?















More





















 

Sign In/Free Sign Up 






*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
S&P 500 Futures
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
Nasdaq Futures
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
Nasdaq Futures
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=

S&P 500
Dow Jones
Nasdaq 100
DAX
FTSE 100
Euro Stoxx 50
Nikkei 225

*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
S&P 500
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
S&P 500
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
Dow Jones
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
Dow Jones
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
Nasdaq 100
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
Nasdaq 100
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
DAX
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
DAX
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
FTSE 100
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
FTSE 100
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
Euro Stoxx 50
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
Euro Stoxx 50
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
Nikkei 225
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
Nikkei 225
*=*=*=*=*=*=*

*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=

*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=

*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
 
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
 Nasdaq12,965.34+27.22+0.21% 
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
 
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
Nasdaq
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
Nasdaq
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
12,965.34
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
+27.22
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
+0.21%
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=

*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=

*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
 
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
 S&P 500 VIX19.560.000.00% 
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
 
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
S&P 500 VIX
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
S&P 500 VIX
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
19.56
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
0.00
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=
0.00%
*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*

In [149]:
import re


In [195]:
(re.search('>\d.*?\d<',re.search('Price to Book.*?\d<', html_doc.replace("\n","ss"))[0])[0]).replace("<","").replace(">","")

'160.69'

In [170]:
html_doc.strip()

'<!DOCTYPE HTML>\n<html dir="ltr" xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" xmlns:schema="http://schema.org/" class="com" lang="en" geo="KR">\n<head>\n<link rel="dns-prefetch" href="https://i-invdn-com.investing.com" />\n<link rel="dns-prefetch" href="https://a-invdn-com.investing.com" />\n<link rel="preconnect" href="https://securepubads.g.doubleclick.net" />\n<link rel="dns-prefetch" href="https://securepubads.g.doubleclick.net" />\n<script async src="https://securepubads.g.doubleclick.net/tag/js/gpt.js"></script>\n<script type="text/javascript">\n    function OptanonWrapper() { }\n\n    // temp for checks\n    </script>\n<style>\n\t.ot-floating-button {\n\t\tz-index:12 !important;\n\t}\n</style>\n<script type="text/javascript">\n        window.helpers = {\n            \'getCookie\': function (k){var v="; "+document.cookie;var p=v.split("; "+k+"=");if(p.length===2)return p.pop().split(";").shift();else return null},\n            \'setCookie\': function (k,v,d){var e=\'\';if(

---

In [294]:
df3

,Prev_Close_Price,Market_Cap,Shares,Beta,Price_To_Book
Ticker,,,,,
LEV,"27,940",-29.59%,-29.59%,"18,939",NaN
LEV.W,NaN,NaN,NaN,NaN,NaN
LEXI,3.1,-0.652,"193,793,551",1.5,8.4
LFEQ,NaN,NaN,NaN,NaN,NaN
LFG,2.2,0.42,"14,112,499",0.41,0.65
...,...,...,...,...,...
ZTEST,0.1,-0.015,"24,227,196",0.647,3.39
ZVIA,NaN,NaN,NaN,NaN,NaN
ZVV,NaN,NaN,NaN,NaN,NaN


CONCAT

In [428]:
df1 = pd.read_csv('factor_by_ticker0_3768.csv', index_col=0)
df2 = pd.read_csv('factor_by_ticker3768_5786.csv', index_col=0)
df3 = pd.read_csv('factor_by_ticker5788_.csv', index_col=0)

In [429]:
df1 = df1[:3769]
df2 = df2[:2019]
df3 = df3[:-1]

In [430]:
df = pd.concat([df1, df2, df3])

전처리 해야할 부분
- Prev_Close_Price
    - 1000 이상의 값 1,000 표기
- Market_Cap
    - 들어가면 안되는 % 표기
    - 1000000 이상의 값 K,M,B 표기
- Shares
    - 들어가면 안되는 % 표기
    - 1000 이상의 값 1,000 표기
- Beta
    - 1000 이상의 값 1,000 표기
- Price_To_Book
    - 정규표현 잘못되서 들어간 string

In [431]:
df

,Prev_Close_Price,Market_Cap,Shares,Beta,Price_To_Book
Ticker,,,,,
AACI,9.93,0,"20,709,500","31,760",NaN
AACIU,9.98,0,"15,000,000",-42.38,NaN
AACIW,NaN,NaN,NaN,NaN,NaN
AADI,13.14,-7.5,"21,016,833",1.59,2.56
AADR,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
ZSL,NaN,NaN,NaN,NaN,NaN
ZTEST,0.1,-0.015,"24,227,196",0.647,3.39
ZVIA,NaN,NaN,NaN,NaN,NaN


In [432]:
df.Prev_Close_Price = df.Prev_Close_Price.apply(lambda x: float(str(x).replace(',', '')))

df.loc[df.Shares.str.contains('%', na=False), 'Shares'] = np.nan

df.Shares = df.Shares.apply(lambda x: float(str(x).replace(',', '')))
df.loc[df.Market_Cap.str.contains('%', na=False), 'Market_Cap'] = np.nan
df.loc[df.Market_Cap.str.contains('M', na=False), 'Market_Cap'] = df.loc[df.Market_Cap.str.contains('M', na=False), 'Market_Cap'].apply(lambda x: 1000000*float(x.replace('M', '')))
df.loc[df.Market_Cap.str.contains('B', na=False), 'Market_Cap'] = df.loc[df.Market_Cap.str.contains('B', na=False), 'Market_Cap'].apply(lambda x: 1000000000*float(x.replace('B', '')))
df.loc[df.Market_Cap.str.contains('K', na=False), 'Market_Cap'] = df.loc[df.Market_Cap.str.contains('K', na=False), 'Market_Cap'].apply(lambda x: 1000*float(x.replace('K', '')))
df.Market_Cap = df.Market_Cap.apply(lambda x: float(str(x).replace(',', '')))
df.loc[df.Beta.str.contains('K', na=False), 'Beta'] = df.loc[df.Beta.str.contains('K', na=False), 'Beta'].apply(lambda x: 1000*float(x.replace('K', '')))
df.loc[df.Beta.str.contains('M', na=False), 'Beta'] = df.loc[df.Beta.str.contains('M', na=False), 'Beta'].apply(lambda x: 1000000*float(x.replace('M', '')))
df.loc[df.Beta.str.contains('B', na=False), 'Beta'] = df.loc[df.Beta.str.contains('B', na=False), 'Beta'].apply(lambda x: 1000000000*float(x.replace('B', '')))
df.loc[df.Beta.str.contains('%', na=False), 'Beta'] = np.nan

df.Beta = df.Beta.apply(lambda x: float(str(x).replace(',', '')))
df.Price_To_Book = df.Price_To_Book.apply(lambda x: str(x).replace(',', ''))
df.Price_To_Book = df.Price_To_Book.apply(lambda x: np.nan if len(str(x))>8 else float(x))
df['BM_Ratio'] = 1/df['Price_To_Book']

df.info()

In [452]:
df.to_csv('us_stock_factor_nullindb.csv')

In [448]:
df.loc['META']

Prev_Close_Price    1.748500e+02
Market_Cap          1.222000e+01
Shares              2.687548e+09
Beta                1.410000e+00
Price_To_Book       3.730000e+00
Name: META, dtype: float64

In [407]:
df

,Prev_Close_Price,Market_Cap,Shares,Beta,Price_To_Book
Ticker,,,,,
AACI,9.93,0,"20,709,500","31,760",NaN
AACIU,9.98,0,"15,000,000",-42.38,NaN
AACIW,NaN,NaN,NaN,NaN,NaN
AADI,13.14,-7.5,"21,016,833",1.59,2.56
AADR,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
ZTEST,0.1,-0.015,"24,227,196",0.647,3.39
ZVIA,NaN,NaN,NaN,NaN,NaN
ZVV,NaN,NaN,NaN,NaN,NaN


In [320]:
df.loc['META']

Prev_Close_Price           174.85
Market_Cap                  12.22
Shares              2,687,548,472
Beta                         1.41
Price_To_Book                3.73
Name: META, dtype: object

In [402]:
df.to_csv('us_stock_factor_nullindb.csv')

In [399]:
pd.read_csv('us_stock_db.csv')

,PROD_CODE
0,US46090E1038
1,US0378331005
2,US22544F1030
3,US22266T1097
4,US00214Q3020
...,...
298,US8288061091
299,US78468R1014
300,US92204A5048
301,US00214Q2030
